In [1]:
import cv2
import os
import numpy as np
import json
from utils.augment import DataAugmentForObjectDetection

In [2]:
path = '../datasets/tt100k/test_new'
image_path = os.path.join(path,'3236.jpg')
annotations_path = os.path.join(path,'annotations.json')
image_id = '3236'

In [3]:
annotations = json.loads(open(annotations_path).read())
types = annotations['types']
labels = []
for obj in annotations['imgs'][image_id]['objects']:
    category = obj['category']
    category_id = types.index(category)
    xmin = int(obj['bbox']['xmin'])
    ymin = int(obj['bbox']['ymin'])
    xmax = int(obj['bbox']['xmax'])
    ymax = int(obj['bbox']['ymax'])
    label = [xmin,ymin,xmax,ymax,category_id]
    labels.append(label)
labels = np.array(labels)
print(labels)
print(labels.shape)
bboxes = labels[:,:4].tolist()
print(bboxes)
print(len(bboxes))

[[132 480 167 519  71]
 [170 481 202 515  91]
 [204 481 240 516  22]
 [242 481 278 517 108]
 [687 523 712 556  71]]
(5, 5)
[[132, 480, 167, 519], [170, 481, 202, 515], [204, 481, 240, 516], [242, 481, 278, 517], [687, 523, 712, 556]]
5


In [4]:
img = cv2.imread(image_path)
aug = DataAugmentForObjectDetection()
img_new,bboxes_new = aug.letterbox(img,bboxes)
print(img_new.shape)
print(len(bboxes_new))
print(bboxes_new)

(416, 416, 3)
5
[[128, 136, 138, 147], [139, 137, 148, 146], [149, 137, 159, 147], [159, 137, 170, 147], [286, 149, 293, 158]]


In [6]:
for bbox in bboxes_new:
    x_min = bbox[0]
    y_min = bbox[1]
    x_max = bbox[2]
    y_max = bbox[3]
    draw = cv2.rectangle(img_new,(int(x_min), int(y_min)), (int(x_max), int(y_max)), (0, 255, 0), 1)

In [ ]:
cv2.namedWindow('pic', 0)  # 1表示原图
cv2.moveWindow('pic', 0, 0)
cv2.resizeWindow('pic', 800, 800)  # 可视化的图片大小
cv2.imshow('pic', draw)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [2]:
import numpy as np
import torch
from torch.autograd import Variable

In [12]:
list = []
a = np.array(list)
print(a)

[]


In [13]:
b = Variable(torch.from_numpy(a).type(torch.FloatTensor))
print(b)

tensor([])


In [18]:
print(b.shape)

torch.Size([0])


In [21]:
for t in range(0):
    print(t)

In [6]:
str = '10056.jpg'
line=str.split()
# box = np.array([np.array(list(map(int, box.split(',')))) for box in line[1:]])

In [14]:
for box in line[1:]:
    box.split(',')
    print(box)

In [3]:
from utils.dataloader import random_crop,random_perspective,letterbox,augment_hsv,box_candidates

In [4]:
def draw(image,bboxes):
    for bbox in bboxes:
        x_min = bbox[0]
        y_min = bbox[1]
        x_max = bbox[2]
        y_max = bbox[3]
        draw = cv2.rectangle(image,(int(x_min), int(y_min)), (int(x_max), int(y_max)), (0, 255, 0), 1)
    cv2.namedWindow('pic', 0)  # 1表示原图
    cv2.moveWindow('pic', 0, 0)
    cv2.resizeWindow('pic', 800, 800)  # 可视化的图片大小
    cv2.imshow('pic', draw)
    cv2.waitKey(0)
    cv2.destroyAllWindows()

In [5]:
with open('tt100k_train.txt') as f:
    lines = f.readlines()

In [6]:
print(lines[0])

C:/Users/Peter/PycharmProjects/datasets/tt100k\train/10009.jpg 1296,932,1319,957,4



In [9]:
line = lines[0].split()
image = cv2.imread(line[1])
targets = np.array([np.array(list(map(int, box.split(',')))) for box in line[1:]])  # xyxy,cls
n = len(targets)
bboxes = targets[:, :4]
cls = targets[:, -1].reshape(n,1)
print(targets)
print(bboxes)
print(cls)

[[1296  932 1319  957    4]]
[[1296  932 1319  957]]
[[4]]


In [10]:
# 1、随机裁剪，更新image、bboxes和targets
image, bboxes = random_crop(image, bboxes)
draw(image,bboxes)
targets = np.concatenate((cls,bboxes),axis = 1)
print(targets)

AttributeError: 'NoneType' object has no attribute 'shape'

In [ ]:

# 1、随机裁剪，更新image、bboxes和targets
image, bboxes = random_crop(image, bboxes)
targets = np.concatenate((cls, bboxes), axis=1)

# 2、letterbox，输出416x416
image, targets = letterbox(image, targets, new_shape=input_shape)

# 3、随机透视变换
image, targets = random_perspective(image, targets)

# 4、色域变换
augment_hsv(image, hgain=hgain, sgain=sgain, vgain=vgain)

# 5、targets由cls,xyxy转为xyxy,cls
cls = targets[:, 0]
bboxes = targets[:, 1:5]
targets = np.concatenate((bboxes, cls), axis=1)